In [1]:
require(tidyverse)
require(data.table)
require(forestplot)
require(meta)


Loading required package: tidyverse
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.0       ✔ purrr   0.3.1  
✔ tibble  2.0.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: data.table

Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last

The following object is masked from ‘package:purrr’:

    transpose

Loading required package: forestplot
Loading required package: grid
Loading required package: magrittr

Attaching package: ‘magrittr’

The following object is masked from ‘package:purrr’:

    set_names

The following object is masked from ‘package:tidyr’:

    extract

Loading required package: checkmate
Loading required package: meta
Loadi

In [ ]:
#CHROM  POS     ID      BETA    SE
136 1       11252357        rs200058074     -0.0380036      0.111454
140 1       11252369        rs28991002      -0.155464       0.0483432
175 1       11253684        rs28991009      -0.200253       0.0270947
177 1       11253688        rs143435072     -0.262552       0.126593

In [2]:
madata <- data.frame(
    Variant = c('Gln175His', 'Arg177Ter', 'Arg140His', 'Gln136Arg'),
    TE      = c(-0.200253,  -0.262552, -0.155464,  -0.0380036), 
    seTE    = c( 0.0270947,  0.126593,  0.0483432,  0.111454),
    stringsAsFactors=F
)


In [3]:
m<-metagen(
    TE,
    seTE,
    data=madata,
    studlab=paste(Variant),
    comb.fixed = TRUE,
    comb.random = FALSE,
    prediction=TRUE,
    sm="SMD"
)

In [4]:
m

              SMD             95%-CI %W(fixed)
Gln175His -0.2003 [-0.2534; -0.1471]      70.5
Arg177Ter -0.2626 [-0.5107; -0.0144]       3.2
Arg140His -0.1555 [-0.2502; -0.0607]      22.1
Gln136Arg -0.0380 [-0.2564;  0.1804]       4.2

Number of studies combined: k = 4

                        SMD             95%-CI     z  p-value
Fixed effect model  -0.1856 [-0.2302; -0.1410] -8.16 < 0.0001
Prediction interval         [-0.2835; -0.0877]               

Quantifying heterogeneity:
tau^2 = 0; H = 1.00 [1.00; 2.47]; I^2 = 0.0% [0.0%; 83.6%]

Test of heterogeneity:
    Q d.f. p-value
 2.80    3  0.4228

Details on meta-analytical method:
- Inverse variance method

In [5]:
m$pval.fixed

[1] 3.360199e-16

In [6]:
pooled_TE   = m$TE.fixed
pooled_seTE = m$seTE.fixed

In [12]:
tabletext<-cbind(
    c('Variant',    madata %>% pull(Variant), 'Summary'),
    c('SD(IOP)', madata %>% pull(TE) %>% lapply(round, digits =2) %>% lapply(as.character) %>% 
      simplify(), as.character(round(pooled_TE, digits=2)))
) 


In [13]:
plot_data <- bind_rows(
    data.frame(TE=NA, seTE=NA), 
    madata, 
    data.frame(TE=pooled_TE, seTE=pooled_seTE)
) %>%
rename(mean = TE) %>%
mutate(
    lower = mean - 1.96 * seTE,
    upper = mean + 1.96 * seTE
) %>%
select(mean, lower, upper)

In [14]:
png(file='ANGPTL7_IOP-normalized_forest.png', width=1600, height=1200, res=300) 
forestplot(
    tabletext, 
    plot_data,
    new_page = TRUE,
    is.summary=c(TRUE,rep(FALSE,madata %>% dim() %>% first()),TRUE),
    xlog=FALSE, 
    col=fpColors(box="royalblue",line="darkblue", summary="royalblue")
)
dev.off()

png 
  2

## reference 
- https://bookdown.org/MathiasHarrer/Doing_Meta_Analysis_in_R/fixed.html
- https://cran.r-project.org/web/packages/forestplot/vignettes/forestplot.html